# Data and Constants

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from scipy.stats import stats
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.stats import norm
import matplotlib.pyplot as plt
from pycaret.regression import *
from multiprocessing import Pool, cpu_count
from concurrent.futures import ThreadPoolExecutor

In [2]:
lipid_path = 'data/section12/lipids_section_12.h5'
gene_path = 'data/section12/genes_section_12.h5'

# Helper Functions

In [3]:
def gaussian_weight(distance, std_dev):
    return norm.pdf(distance, 0, std_dev)

In [4]:
def exponential_decay(distance, threshold, decay_rate):
    adjusted_distance = distance - threshold
    return np.exp(-decay_rate * adjusted_distance)

In [5]:

def filter_dataframe_from_file(df, filepath):
    # Read the text file and store the entries in a list
    with open(filepath, 'r') as file:
        entries = file.read().splitlines()

    # Filter the DataFrame based on the index matching the entries
    filtered_df = df.loc[df.index.isin(entries)]

    return filtered_df

# Loading Dataset

In [6]:
# Loading the dataset
lipids_section_12 = pd.read_hdf(lipid_path)
genes_section_12 = pd.read_hdf(gene_path)

# Need to remove the trailing naming scheme added before
new_column_names = [re.sub(r'_(\d+)$', '', col) for col in lipids_section_12.columns]
lipids_section_12.columns = new_column_names

# Create cKDTree for fast query of neighbors

In [7]:
# Create a KDTree object for the genes
genes_coords = genes_section_12[['y_ccf', 'z_ccf']].values
genes_kdtree = cKDTree(genes_coords)

# Extract coordinates for lipids
lipids_coords = lipids_section_12[['y_ccf', 'z_ccf']].values

# Nearest Neighbor selection

In [7]:
# Find the indices of the closest gene for each lipid point
_, indices = genes_kdtree.query(lipids_coords, k=1)

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), genes_section_12.iloc[:, 46:-50].shape[1]))

# Aggregate gene data based on the closest neighbor
for i, gene_index in enumerate(indices):
    aggregated_gene_data[i] = genes_section_12.iloc[gene_index, 46:-50]

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Average of nearest neighbors

In [44]:
# Query to get the 5000 closest genes for each lipid point
_, indices = genes_kdtree.query(lipids_coords, k=10000)

# Pre-allocate memory for aggregated gene data
n_genes = genes_section_12.iloc[:, 46:-50].shape[1]
aggregated_gene_data = np.zeros((len(lipids_coords), n_genes))

# Define a function to aggregate data
def aggregate_data(i):
    gene_indices = indices[i]
    data = genes_section_12.iloc[gene_indices, 46:-50].values
    return data.mean(axis=0)

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.map(aggregate_data, range(len(lipids_coords)))

# Assign the results back to the array
for i, res in enumerate(results):
    aggregated_gene_data[i] = res

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Exponential Decay

In [14]:
# Find the distances and indices of the 6 closest genes for each lipid point
distances, indices = genes_kdtree.query(lipids_coords, k=6)

# Calculate the average distance
average_closest_distance = np.mean(distances)

# Query to get the 100 closest genes and their distances
distances, indices = genes_kdtree.query(lipids_coords, k=1000)

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), genes_section_12.iloc[:, 46:-50].shape[1]))

# Perform weighted aggregation
for i, (gene_indices, dists) in enumerate(zip(indices, distances)):
    # Weights based on distance, with a penalty for distances greater than the average closest distance
    weights = np.where(dists <= average_closest_distance, 1, exponential_decay(dists, average_closest_distance, 3))  # Apply penalty for dist > average_closest_distance
    weighted_data = genes_section_12.iloc[gene_indices, 46:-50] * weights[:, np.newaxis]
    aggregated_gene_data[i] = weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(genes_section_12.iloc[:, 46:-50].shape[1])

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])


# K neighbors gaussian mean of genes for a given lipids datapoint

In [8]:
# Find the distances and indices of the closest 2500 genes for each lipid point
distances, _ = genes_kdtree.query(lipids_coords, k=2500)

# Calculate the std of the distances
std_closest_distance = np.std(distances, axis=1)

# Query to get the 5000 closest genes and their distances
distances, indices = genes_kdtree.query(lipids_coords, k=5000)

# Pre-allocate memory for aggregated gene data
n_genes = genes_section_12.iloc[:, 46:-50].shape[1]
aggregated_gene_data = np.zeros((len(lipids_coords), n_genes))

# Define a function to aggregate data
def aggregate_data(i):
    dists = distances[i]
    gene_indices = indices[i]
    weights = gaussian_weight(dists, std_closest_distance[i])
    weighted_data = genes_section_12.iloc[gene_indices, 46:-50].values * weights[:, np.newaxis]
    return weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(n_genes)

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.map(aggregate_data, range(len(lipids_coords)))

# Assign the results back to the array
for i, res in enumerate(results):
    aggregated_gene_data[i] = res

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=genes_section_12.columns[46:-50])

# Take all the points into consideration

In [8]:
def process_lipid_point(lipid_coord, genes_coords, gene_data, std_closest_distance):
    """Process a single lipid point and return the aggregated data."""
    # Calculate distances to all gene points
    dists = np.linalg.norm(genes_coords - lipid_coord, axis=1)

    # Weights based on distance, with a penalty for distances greater than the average closest distance
    weights = gaussian_weight(dists, std_closest_distance)

    # Perform weighted aggregation
    weighted_data = gene_data * weights[:, np.newaxis]
    return weighted_data.sum(axis=0) / weights.sum() if weights.sum() > 0 else np.zeros(gene_data.shape[1])

In [9]:

gene_data_columns = genes_section_12.columns[46:-50]
gene_data_shape = genes_section_12[gene_data_columns].shape[1]

# Initialize an empty array for aggregated gene data
aggregated_gene_data = np.zeros((len(lipids_coords), gene_data_shape))

distances, _ = genes_kdtree.query(lipids_coords, k=6)
std_closest_distance = np.std(distances)

# Extract gene data only once
gene_data = genes_section_12[gene_data_columns].values

# Parallel processing
with Pool(cpu_count()) as pool:
    results = pool.starmap(process_lipid_point, [(lipid_coord, genes_coords, gene_data, std_closest_distance) for lipid_coord in lipids_coords])

# Combine results into aggregated_gene_data
for i, result in enumerate(results):
    aggregated_gene_data[i] = result

# Convert the aggregated data into a DataFrame
aggregated_gene_data_df = pd.DataFrame(aggregated_gene_data, columns=gene_data_columns)


# Resulting genes and lipids DataFrames

In [9]:
aggregated_gene_data_df


,ENSMUST00000028118,ENSMUST00000028280,ENSMUST00000030676,ENSMUST00000047328,ENSMUST00000057021,ENSMUST00000090697,ENSMUST00000091554,ENSMUST00000162772,ENSMUST00000021284,ENSMUST00000022195,...,ENSMUST00000109964,ENSMUST00000114553,ENSMUST00000152412,ENSMUST00000159365,ENSMUST00000175965,ENSMUST00000196378,ENSMUST00000228095,ENSMUST00000000219,ENSMUST00000035577,ENSMUST00000060943
0,0.124094,0.291178,0.128066,0.623594,0.242590,0.074589,0.084546,0.082760,0.323102,0.056296,...,0.028234,0.008777,0.023342,0.219471,0.014906,0.242949,0.012779,0.019391,0.035380,0.037051
1,0.123617,0.291195,0.132069,0.619720,0.238866,0.078791,0.087023,0.086119,0.317896,0.055298,...,0.028362,0.008728,0.025231,0.221363,0.015748,0.246850,0.012993,0.019144,0.036661,0.038754
2,0.122475,0.290745,0.136301,0.614887,0.234399,0.082712,0.089679,0.089638,0.312165,0.054260,...,0.028319,0.008792,0.027318,0.223340,0.016716,0.249978,0.013238,0.019240,0.037741,0.040391
3,0.120780,0.290199,0.140761,0.609180,0.229255,0.086308,0.092639,0.093405,0.305956,0.053218,...,0.028152,0.008972,0.029628,0.225443,0.017806,0.252181,0.013485,0.019686,0.038572,0.042067
4,0.118690,0.289892,0.145317,0.602782,0.223699,0.089497,0.095860,0.097383,0.299470,0.052184,...,0.027889,0.009264,0.032130,0.227648,0.018977,0.253397,0.013693,0.020454,0.039100,0.043767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94742,0.170155,0.380321,0.356490,0.537902,0.281510,0.095585,0.233756,0.189178,0.395402,0.047084,...,0.015559,0.061364,0.013411,0.123024,0.035987,0.144793,0.012435,0.033792,0.030281,0.008782
94743,0.169232,0.385538,0.349213,0.542164,0.289974,0.091108,0.232701,0.176244,0.399768,0.044931,...,0.014296,0.059693,0.012953,0.125018,0.036701,0.143519,0.011728,0.031828,0.029679,0.007909
94744,0.168109,0.391031,0.341711,0.546342,0.298262,0.086795,0.231340,0.165354,0.403440,0.042890,...,0.013114,0.058164,0.012628,0.127244,0.037463,0.141764,0.011042,0.029889,0.029413,0.007109
94745,0.166513,0.395388,0.338151,0.544088,0.299778,0.083737,0.233229,0.159442,0.401881,0.041783,...,0.012161,0.057034,0.012614,0.129440,0.038170,0.138102,0.010601,0.028355,0.029906,0.006518


In [10]:
section_12_lipids_only = lipids_section_12.iloc[:, 3:-3]
section_12_lipids_only

,LPC O-16:2,LPC 16:0_dup,LPC O- 18:3,LPC O-18:2,LPC O-16:2_dup,LPC 15:1,LPC 18:1,LPC 18:0_dup,LPC 16:0,LPC O-18:3,...,SM(t42:1),PC(40:7),PC 40:6_dup,PG(42:6),Hex2Cer 32:0,SHexCer 38:1;3,PE(44:11(OH)),PC(40:4),PS(40:4),PIP(O-36:5)
section12_pixel23_121,0.000140,0.000112,0.000116,0.000125,0.000214,0.000100,0.0001,0.000197,0.000179,0.0001,...,0.0001,0.000100,0.000241,0.000179,0.0001,0.000100,0.0001,0.000261,0.0001,0.000360
section12_pixel23_122,0.000213,0.000112,0.000114,0.000125,0.000204,0.000162,0.0001,0.000100,0.000181,0.0001,...,0.0001,0.000114,0.000395,0.000208,0.0001,0.000316,0.0001,0.000268,0.0001,0.000100
section12_pixel23_123,0.000154,0.000100,0.000117,0.000134,0.000195,0.000151,0.0001,0.000232,0.000179,0.0001,...,0.0001,0.000114,0.000233,0.000203,0.0001,0.000100,0.0001,0.000232,0.0001,0.000100
section12_pixel23_124,0.000147,0.000113,0.000114,0.000136,0.000229,0.000154,0.0001,0.000100,0.000120,0.0001,...,0.0001,0.000114,0.000285,0.000187,0.0001,0.000255,0.0001,0.000100,0.0001,0.000366
section12_pixel23_125,0.000229,0.000112,0.000115,0.000206,0.000100,0.000100,0.0001,0.000100,0.000122,0.0001,...,0.0001,0.000100,0.000247,0.000179,0.0001,0.000323,0.0001,0.000100,0.0001,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
section12_pixel308_157,0.000139,0.000100,0.000100,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_158,0.000139,0.000100,0.000100,0.000100,0.000100,0.000100,0.0001,0.000100,0.000119,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_159,0.000155,0.000112,0.000117,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100
section12_pixel308_160,0.000141,0.000113,0.000100,0.000100,0.000100,0.000100,0.0001,0.000237,0.000119,0.0001,...,0.0001,0.000100,0.000100,0.000100,0.0001,0.000100,0.0001,0.000100,0.0001,0.000100


In [11]:
aggregated_gene_data_df = aggregated_gene_data_df.reset_index(drop=True)
section_12_lipids_only = section_12_lipids_only.reset_index(drop=True)

# Simplify lipid names

In [12]:
def rename_lipid(lipid_name):
    # Example renaming scheme - customize as needed
    new_name = lipid_name.replace(':', ';').replace('\xa0', ' ')
    return new_name

section_12_lipids_only.columns = [rename_lipid(lipid) for lipid in section_12_lipids_only.columns] 

# Split Train/Test set

In [16]:
# Prepare the features and target dataframes
features_df = aggregated_gene_data_df.copy()
target_df = section_12_lipids_only.copy()

# Rename the columns of target_df to ensure uniqueness
unique_column_names = [f'{name}_{i}' for i, name in enumerate(target_df.columns)]
target_df.columns = unique_column_names

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.3, random_state=42)

# Export the full training and test sets to .h5 files
X_train.to_hdf('data/train_features.h5', key='X_train', mode='w')
X_test.to_hdf('data/test_features.h5', key='X_test', mode='w')
y_train.to_hdf('data/train_targets.h5', key='y_train', mode='w')
y_test.to_hdf('data/test_targets.h5', key='y_test', mode='w')

# Restoring the names
new_column_names = [re.sub(r'_(\d+)$', '', col) for col in target_df.columns]
target_df.columns = new_column_names

# Selecting best model and training it and see the results

In [ ]:
for i in range(len(y_train.columns)):
    # Extract the column name for the current index
    lipid_name = y_train.columns[i]

    # Concatenate the lipid column with the training and testing features
    train_data = pd.concat([X_train, y_train.iloc[:, i]], axis=1)
    test_data = pd.concat([X_test, y_test.iloc[:, i]], axis=1)
    
    # Setup PyCaret for each lipid
    # Ensure that the test dataset is correctly specified
    setup(data=train_data, test_data=test_data, 
          fold=5, session_id=42, use_gpu=True, verbose=False, preprocess=False)

    # Create and plot the model
    model = create_model('catboost')
    plot_model(model, plot='feature')

# Evaluating One Model using K-Cross Validation

In [19]:
results_df = pd.DataFrame(columns=['Lipid', 'R2_mean', 'MAPE_mean'])

for i, lipid_name in enumerate(y_train.columns):
    # Extract the column name for the current index
    lipid_name = y_train.columns[i]

    # Concatenate the lipid column with the training and testing features
    train_data = pd.concat([X_train, y_train.iloc[:, i]], axis=1)
    test_data = pd.concat([X_test, y_test.iloc[:, i]], axis=1)
    
    # Setup PyCaret for each lipid
    # Ensure that the test dataset is correctly specified
    setup(data=train_data, test_data=test_data, 
          fold=5, session_id=42, use_gpu=True, verbose=False, preprocess=False)
    
    # Create and plot the model
    model = create_model('catboost')
    
    # Retrieving cross-validation results
    metrics = pull()
    
    r2_mean = metrics.loc['Mean','R2']
    mape_mean = metrics.loc['Mean', 'MAPE']
    
    print("R2 mean: ", r2_mean)
    print("MAPE mean: ", mape_mean)
    
    # Append the results to the DataFrame
    results_df = results_df.append({'Lipid': lipid_name, 'R2_mean': r2_mean, 'MAPE_mean': mape_mean}, ignore_index=True)

results_df

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

...

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:55:40
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Regressor


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
results_df.to_csv('results.csv')